# Semantic Search for Songs App

## Table of Contents

1. Overview
2. The Model
3. Set Up Qdrant Cloud
4. Load Vectors

In [12]:
from qdrant_client import QdrantClient, models
import pandas as pd
import numpy as np
import os, ast

In [17]:
collection = "music_vectors"
batch_size = 250


client = QdrantClient(
    "https://6d917786-6584-44ae-839d-9893107d9a95.us-east4-0.gcp.cloud.qdrant.io",
    api_key=os.environ['QDRANT_API_KEY']
)

In [13]:
metadata = pd.read_csv('payload.csv')
metadata['subgenres'] = metadata['subgenres'].apply(ast.literal_eval)

In [15]:


embeddings = np.load('audio_vectors.npy')
index = metadata['index'].tolist()
payload = metadata.drop(['index', 'ids'], axis=1).to_dict(orient="records")

In [18]:
client.recreate_collection(
    collection_name=collection,
    vectors_config=models.VectorParams(
        size=embeddings.shape[1], distance=models.Distance.COSINE
    )
)

True

In [20]:
client.upsert??

Signature:
client.upsert(
    collection_name: str,
    points: Union[qdrant_client.http.models.models.Batch, List[Union[qdrant_client.http.models.models.PointStruct, points_pb2.PointStruct]]],
    wait: bool = True,
    ordering: Optional[qdrant_client.http.models.models.WriteOrdering] = None,
    shard_key_selector: Union[Annotated[int, Strict(strict=True)], Annotated[str, Strict(strict=True)], List[Union[Annotated[int, Strict(strict=True)], Annotated[str, Strict(strict=True)]]], NoneType] = None,
    **kwargs: Any,
) -> qdrant_client.http.models.models.UpdateResult
Source:   
    def upsert(
        self,
        collection_name: str,
        points: types.Points,
        wait: bool = True,
        ordering: Optional[types.WriteOrdering] = None,
        shard_key_selector: Optional[types.ShardKeySelector] = None,
        **kwargs: Any,
    ) -> types.UpdateResult:
        """
        Update or insert a new point into the collection.

        If point with given ID already exists - i

In [22]:
for i in range(0, metadata.shape[0], batch_size):

    low_idx = min(i+batch_size, metadata.shape[0])

    batch_of_ids = index[i: low_idx]
    batch_of_embs = embeddings[i: low_idx]
    batch_of_payloads = payload[i: low_idx]

    client.upsert(
        collection_name=collection,
        points=models.Batch(
            ids=batch_of_ids,
            vectors=batch_of_embs.tolist(),
            payloads=batch_of_payloads
        ),
        wait=True
    )